<h1>Make a Network Graph File</h1>
This code will generate a file in gexf format so that you can visualise your network.
This 'Jupyter Lab' workspace is made up of blocks of python code. To run each block click the 'play' button above.

<h3>1. Load Libraries</h3>

Python, the langaugae we are using here, is made up of libraries that we can use to help us. First we load the libraires we need.

In [ ]:
from io import BytesIO
import pandas as pd
from ipywidgets import FileUpload
import networkx as nx
import math
uploader = FileUpload(accept='.xlsx', multiple=True)

<h3>2. Upload your excel files</h3> 

We are comparing two projects. Run this next block of code then click upload to select your two excel files. If done correctly, the "Upload(0)" button should change to "Upload(2)" to show that you have uploaded 2 files. They must be excel files! 

In [ ]:
uploader

<h3>3. Check your excel file</h3> 

Check the name of the file you have uploaded.

In [ ]:
roles = []
for filename, fileinfo in uploader.value.items():
    content = fileinfo['content']
    try:
        # Read Excel file
        excel_data = pd.read_excel(BytesIO(content))
        
        # Display the dataframe
        print(f"Displaying content of: {filename}")
        display(excel_data)  # For Jupyter Notebook
        roles = roles + list(excel_data['Role'])
        
    except Exception as e:
        print(f"Error reading {filename}: {e}")

<h3>4. Define Colours</h3>
We can colour your graph based on the role of actors. Run the code below to get a list of all the actors in your projects.

In [ ]:
print(set(roles))

In the code below, colours have been given to different roles. 
- Update the list of roles to match your project.
- give them 'rgb' colours.
- Make sure that words match exactly (including uppercase, lowercase, and spaces).
- Be careful not to change the syntax!
- Add more lines if you need to.

In [ ]:
roles = {
    'contractor': {'r': 255, 'g': 0, 'b': 0},
    'client': {'r': 0, 'g': 255, 'b': 0},
    'design_team': {'r': 100, 'g': 0, 'b': 20},
    'architect': {'r': 0, 'g': 100, 'b': 50},
    'authority': {'r': 50, 'g': 50, 'b': 50},
    'consultant': {'r': 75, 'g': 25, 'b': 25},
}

<h3>5. Create your network file</h3>

Run the following block of code to generate a GEXF file for you network. If successful, you should see a 'done!' message appear below the code. If you haven't listed all of your roles, you will get an error!

In [ ]:
def makeGraph(dfActor, dfPoint, filename):

    try:
        dfPoint = dfPoint.drop(['total'])
        dfPoint = dfPoint.drop(columns=['total'])
    except:
        pass
    
    nodeDict = {}
    for k,v in dfActor.iterrows():
        nodeDict['{k}_{g}'.format(k=k, g=filename)] = {
            'name': k,
            'role': v['Role'],
            'weight': [],
        }
    
    edgeDict = {}
    cols = list(dfPoint)
    names = []
    for col in cols:
        try:
            col = float(col)
        except ValueError:
            if col != 'total':
                names.append(col)
   
    for k, v in dfPoint.iterrows():
        if k in names:
            n1 = '{k}_{g}'.format(k=k, g=filename)
            
            for name in names:
                n2 = '{k}_{g}'.format(k=name, g=filename)
                
                if n1 != n2:
                    edge = tuple(sorted([n1, n2]))
                    edgeWeight = v[name]
                          
                    if edge not in edgeDict.keys():
                        edgeDict[edge] = {
                            'weight': []
                        }

                    if math.isnan(edgeWeight) == True:
                        edgeWeight = 0
                    edgeDict[edge]['weight'].append(edgeWeight)   
                    nodeDict[n1]['weight'].append(edgeWeight)
                    nodeDict[n2]['weight'].append(edgeWeight)
 
    return nodeDict, edgeDict

# iterate through stages
for i in range(3):
    sheet = 'Point{n}'.format(n=i+1)
    print(sheet)

    # Make graph
    G = nx.Graph()
    print(' ')
    
    for filename, fileinfo in uploader.value.items():
        content = fileinfo['content']
        dfActor = pd.read_excel(BytesIO(content), sheet_name='Actors', header=0, index_col=0)
        dfPoint = pd.read_excel(BytesIO(content), sheet_name=sheet, header=0, index_col=0)

        nodeDict, edgeDict = makeGraph(dfActor, dfPoint, filename)

        for n in nodeDict.keys():
            try:
                weight = sum(nodeDict[n]['weight'])
                if weight > node_max:
                    node_max = weight
            except:
                pass

        for n in nodeDict.keys():
            try:
                weight = sum(nodeDict[n]['weight'])
            except:
                weight = 0
            name = nodeDict[n]['name']
            role = nodeDict[n]['role']
            G.add_node(n, label=name, role=role, size=weight)
    
        for e in edgeDict.keys():
            try:
                weight = sum(edgeDict[e]['weight'])
            except:
                weight = 0
            edge_viz = {
                'color': {'r': 0, 'g': 0, 'b': 0, 'a': 1.0},
                }
            if weight > 0:
                G.add_edge(e[0], e[1], weight=weight, viz=edge_viz)

    # normalise sizes
    max_size = 8
    min_size = 2

    node_size = []
    # Get all node sizes
    node_sizes = nx.get_node_attributes(G, 'size')
    for k,v in node_sizes.items():
        node_size.append(v)

    for n in G.nodes():
        # Get the node's role and weight
        role = G.nodes[n]['role']
        size = G.nodes[n]['size']
        # Normalize node size
        normalised_size = min_size + (max_size - min_size) * (size - min(node_size)) / (max(node_size) - min(node_size))
        # Create the viz dictionary with color and size
        node_viz = {
            'color': roles[role],  # Default to gray if role is not found
            'size': normalised_size,  # Size based on normalized weight
            'label': {'color': '#000000'}
        }
        # Merge viz attributes into the existing node attributes
        current_viz = G.nodes[n].get('viz', {})
        current_viz.update(node_viz)  # Merge existing viz with new viz attributes
        G.nodes[n]['viz'] = current_viz
        
    max_edge = 5
    min_edge = 1

    edge_weight = []
    # Get all edge sizes
    edge_weights = nx.get_edge_attributes(G, 'weight')
    for k,v in edge_weights.items():
        edge_weight.append(v)

    for e in G.edges():
        # Get the edge's weight
        weight = G.edges[e]['weight']
        # Normalize node size
        normalised_weight = min_edge + (max_edge - min_edge) * (weight - min(edge_weight)) / (max(edge_weight) - min(edge_weight))
        # Update the node with the viz attributes
        G.edges[e]['weight'] = normalised_weight
       
    # write file
    new_filename = '{f}_GEXF.gexf'.format(f=sheet)
    print('{f} created'.format(f=new_filename))
    nx.write_gexf(G, new_filename)
    print(' ')






    

print('done!')

<h3>6. Download your GEXF file!</h3>

Finally, click the 'folder' symbol in the left hand panel to find your GEXF file. Right-click on the file and download to your computer!